In [1]:
import tensorflow as tf
from time import time
import numpy
import os
import json
import pickle
import pandas
import datetime
from functools import partial, reduce

import sys
sys.path.append('../libs')

import data_pipeline
import conv_model
import initialize
import prepare_data
import flacdb

In [2]:
%%time

H = {
    **initialize.load_hypes(), 
    'epochs': 1,
    'steps_per_epoch': 32,
}
initialize.run(H)

92 years,  594510 record segments


/sailhome/kuprel/blood-pressure/env3p7/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/sailhome/kuprel/blood-pressure/env3p7/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


NameError: name 'path' is not defined

In [5]:
data = {}

for part in ['train', 'validation']:
    path = '/scr1/mimic/initial_data_{}_{}.pkl'.format(512, part)
    with open(path, 'rb') as f:
        data[part] = pickle.load(f)

In [ ]:
def write_tensors(data, path, epochs):
    for part in data:
        for k in data[part]:
            file = path + k + '_' + str(epochs) + '_' + part + '.tensor'
            tf.io.write_file(file, tf.io.serialize_tensor(data[part][k]))

In [ ]:
from initialize import TENSOR_INFO

def read_tensors(path, epochs):
    data = {}
    for part in ['train', 'validation']:
        data[part] = {}
        for i in TENSOR_INFO:
            file = path + i['name'] + '_' + str(epochs) + '_' + part + '.tensor'
            t = tf.io.parse_tensor(tf.io.read_file(file), out_type=i['dtype'])
            data[part][i['name']] = t

In [ ]:
H = {
    **initialize.load_hypes(), 
    'input_sigs_train': H0['output_sigs'] + H0['input_sigs_train'], 
    'input_sigs_validation': H0['output_sigs'] + H0['input_sigs_validation'], 
    'epochs': 1,
    'steps_per_epoch': 32,
    'batch_buffer_size': 1,
}

In [ ]:
path = '/scr1/mimic/initial_tensors/'

In [8]:
tf.io.write_file('/scr1/mimic/test.tensor', tf.io.serialize_tensor(t))

In [11]:
[i for i in dir(tf.io) if 'serial' in i]

['deserialize_many_sparse',
 'serialize_many_sparse',
 'serialize_sparse',
 'serialize_tensor']

In [13]:
tf.io.serialize_sparse?

Signature: tf.io.serialize_sparse(sp_input, out_type=tf.string, name=None)
Docstring:
Serialize a `SparseTensor` into a 3-vector (1-D `Tensor`) object.

Args:
  sp_input: The input `SparseTensor`.
  out_type: The `dtype` to use for serialization.
  name: A name prefix for the returned tensors (optional).

Returns:
  A 3-vector (1-D `Tensor`), with each column representing the serialized
  `SparseTensor`'s indices, values, and shape (respectively).

Raises:
  TypeError: If `sp_input` is not a `SparseTensor`.
File:      ~/blood-pressure/env3p7/lib/python3.7/site-packages/tensorflow_core/python/ops/sparse_ops.py
Type:      function
